# Ifript data management

In [77]:
%matplotlib inline

for console comments...

In [78]:
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'
    UNDERLINE = '\033[4m'

In [2]:
import os, fnmatch, zipfile, shutil, sys
import pandas as pd, re


In [3]:
%reload_ext version_information
%version_information pandas, re

Software versions
Python 3.5.2 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 5.2.2
OS Linux 4.9.8 moby x86_64 with glibc2.2.5
pandas 0.19.2
re 2.2.1
Tue Feb 21 22:01:57 2017 UTC

Initialize the variables used.

In [81]:
inDataset=[]
columns_out =['commodity', 'gcm_type', 'impactparameter', 'iso', 'irrigation', 'region', 'scenario', 'value', 'year']


In [82]:
def main(cwd, dirOut, sysargv):
    ensure_dir(cwd+"/dst/")
    dataList = find_files(cwd, sysargv)
    print(dataList)
    print(bcolors.OKBLUE + 'crops' + bcolors.ENDC)
    writeIndicator('combined_prepared', dataList, dirOut)
    print(bcolors.OKGREEN+bcolors.UNDERLINE+'Done, EXIT'+ bcolors.ENDC)

In [83]:
def ensure_dir(f):
    d = os.path.dirname(f)
    if not os.path.exists(d):
        os.mkdir(d)

In [84]:
def find_files(directory, name):
    for root, dirs, files in os.walk(directory):
        for basename in files:
            if fnmatch.fnmatch(basename, name):
                filesource = os.path.join(root, basename)
                base_prnt = (basename.rsplit('.')[0]).rsplit('_')
                item = {'filePath': filesource, 'indicator':base_prnt[0].lower()}
                if len(item) > 0:
                    inDataset.append(item)
    return inDataset

In [85]:
def writeIndicator(indicatorName, inDatas, dirOut):
    frames = []
    outData = dirOut + indicatorName +'.csv'
    for data in inDatas:
        print(data['indicator'])
        x = openXls(data['filePath'], data['indicator'])
        frames.append(x)
    f= pd.concat(frames)
    f.to_csv(outData)
    print(bcolors.OKGREEN + "SUCCESS" + bcolors.ENDC)

In [86]:
def openXls(filePath, indicator):
    with pd.ExcelFile(filePath) as xls:
        data = pd.read_excel(xls, 0)
        data.rename(columns={'impactparameter': 'impactparameter', 'scenario': 'scenario', 'commodity':'commodity', 'region':'region', 'productiontype':'irrigation', 'year':'year', 'Value':'value'}, inplace=True)
        data["gcm_type"]=indicator
        data["iso"]=None
        response = pd.DataFrame(data, columns=columns_out)
        return response

```python
# party time
if __name__ == "__main__":

    # check for correct args: data location
    if len(sys.argv) < 4:
        print "python main.py 'dirIn' 'dirOut' '*.xlsx'"
        exit()

    if os.path.exists(sys.argv[2]):
        shutil.rmtree(sys.argv[2])

    main(sys.argv[1], sys.argv[2], sys.argv[3])
```


In [87]:
main('/home/jovyan/work/data/aqueduct/data_source/compose_country_crops','/home/jovyan/work/data/aqueduct/dst/', '*.xlsx')

[{'indicator': 'gfdl', 'filePath': '/home/jovyan/work/data/aqueduct/data_source/compose_country_crops/gfdl_all.xlsx'}, {'indicator': 'hgem', 'filePath': '/home/jovyan/work/data/aqueduct/data_source/compose_country_crops/hgem_all.xlsx'}, {'indicator': 'ipsl', 'filePath': '/home/jovyan/work/data/aqueduct/data_source/compose_country_crops/ipsl_all.xlsx'}, {'indicator': 'miro', 'filePath': '/home/jovyan/work/data/aqueduct/data_source/compose_country_crops/miro_all.xlsx'}, {'indicator': 'nocc', 'filePath': '/home/jovyan/work/data/aqueduct/data_source/compose_country_crops/NoCC_all.xlsx'}]
crops
gfdl
hgem
ipsl
miro
nocc
SUCCESS
Done, EXIT
